# Week 14: CNN Lab - Rock, Paper, Scissors

**Objective:** Build, train, and test a Convolutional Neural Network (CNN) to classify images of hands playing Rock, Paper, or Scissors.

### Step 1: Setup and Data Download

This first cell downloads the dataset from Kaggle.

In [3]:
!pip install kagglehub

import kagglehub

path = kagglehub.dataset_download("drgfreeman/rockpaperscissors")

print("Path to dataset files:", path)

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\mohdy\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 306M/306M [05:54<00:00, 904kB/s]  

Extracting files...


Path to dataset files: C:\Users\mohdy\.cache\kagglehub\datasets\drgfreeman\rockpaperscissors\versions\2


In [4]:
import shutil
import os

# Determine where the dataset was downloaded/extracted
# Prefer the `path` variable created in the previous cell (kagglehub.dataset_download)
try:
    downloaded_path = path
except NameError:
    downloaded_path = None

# Try to find a directory that contains rock/, paper/, scissors/
src_root = None
if downloaded_path and os.path.exists(downloaded_path):
    for root, dirs, files in os.walk(downloaded_path):
        dirs_lower = [d.lower() for d in dirs]
        if all(x in dirs_lower for x in ['rock', 'paper', 'scissors']):
            src_root = root
            break

# Fallback to common Kaggle input path (if running in Kaggle environment)
if src_root is None:
    candidate = '/kaggle/input/rockpaperscissors'
    if os.path.exists(candidate):
        src_root = candidate

# If downloaded_path itself contains the folders, use it
if src_root is None and downloaded_path:
    if all(os.path.isdir(os.path.join(downloaded_path, d)) for d in ['rock', 'paper', 'scissors']):
        src_root = downloaded_path

if src_root is None:
    print('Dataset folders not found. Please check the downloaded path:', downloaded_path)
else:
    dst_root = os.path.join(os.getcwd(), 'dataset')
    os.makedirs(dst_root, exist_ok=True)

    folders_to_copy = ['rock', 'paper', 'scissors']

    for folder in folders_to_copy:
        src_path = os.path.join(src_root, folder)
        dst_path = os.path.join(dst_root, folder)

        if os.path.exists(src_path):
            shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
            print('Copied:', folder)
        else:
            print('Folder not found in src_root:', folder, 'src_root=', src_root)



Copied: rock
Copied: paper
Copied: scissors


### Step 2: Imports and Device Setup

Import the necessary libraries and check if a GPU is available.

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np

# TODO: Set the 'device' variable
# Check if CUDA (GPU) is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Using device:", device)

Using device: cpu


### Step 3: Data Loading and Preprocessing

Here we will define our image transformations, load the dataset, split it, and create DataLoaders.

In [6]:
DATA_DIR = os.path.join(os.getcwd(), 'dataset')  # local dataset folder placed next to the notebook


# TODO: Define the image transforms
# We need to:
# 1. Resize all images to 128x128
# 2. Convert them to Tensors
# 3. Normalize them (mean=0.5, std=0.5)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Load dataset using ImageFolder
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)

class_names = full_dataset.classes
print("Classes:", class_names)
# Number of classes detected in the dataset (used to size the final layer)
NUM_CLASSES = len(class_names)
print("Detected number of classes:", NUM_CLASSES)
# Print counts per class to help debug unexpected folders
from collections import Counter
labels = [y for _, y in full_dataset.samples]
label_counts = Counter(labels)
for idx, name in enumerate(class_names):
    print(f"  {name}: {label_counts.get(idx, 0)} images")

# Split the dataset: 80% train, 20% test
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])


# Create the DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Test images: {len(test_dataset)}")

Classes: ['paper', 'rock', 'scissors']
Detected number of classes: 3
  paper: 712 images
  rock: 726 images
  scissors: 750 images
Total images: 2188
Training images: 1750
Test images: 438


### Step 4: Define the CNN Model

Fill in the `conv_block` and `fc_block` with the correct layers.

In [7]:
class RPS_CNN(nn.Module):
    def __init__(self):
        super(RPS_CNN, self).__init__()

        # Convolutional feature extractor (3 blocks)
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        # After 3 MaxPool(2) layers, our 128x128 image becomes:
        # 128 -> 64 -> 32 -> 16
        # So the flattened size is 64 * 16 * 16

        # Fully-connected classifier
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, NUM_CLASSES),
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x

# Initialize the model, criterion, and optimizer
model = RPS_CNN().to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

RPS_CNN(
  (conv_block): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=3, bias=True)
  )
)


### Step 5: Train the Model

Fill in the core training steps inside the loop.

In [8]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train() # Set the model to training mode
    total_loss = 0

    for images, labels in train_loader:
        # Move data to the correct device
        images, labels = images.to(device), labels.to(device)

        # 1. Clear the gradients
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(images)

        # 3. Compute loss
        loss = criterion(outputs, labels)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss = {total_loss/len(train_loader):.4f}")

print("Training complete!")

Epoch 1/10, Loss = 0.7204
Epoch 2/10, Loss = 0.2346
Epoch 3/10, Loss = 0.1004
Epoch 4/10, Loss = 0.0489
Epoch 5/10, Loss = 0.0206
Epoch 6/10, Loss = 0.0326
Epoch 7/10, Loss = 0.0550
Epoch 8/10, Loss = 0.0184
Epoch 9/10, Loss = 0.0135
Epoch 10/10, Loss = 0.0047
Training complete!


### Step 6: Evaluate the Model

Test the model's accuracy on the unseen test set.

In [9]:
model.eval() # Set the model to evaluation mode
correct = 0
total = 0

# We don't need to calculate gradients during evaluation
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Get model outputs
        outputs = model(images)

        # Predicted class is the index with the highest logit
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 98.40%


### Step 7: Test on a Single Image

Let's see how the model performs on one image.

In [10]:
def predict_image(model, img_path):
    model.eval()

    img = Image.open(img_path).convert("RGB")
    # Apply the same transforms as training, and add a batch dimension (unsqueeze)
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        # Get the raw model outputs (logits)
        output = model(img)

        # 2. Get the predicted class index
        _, pred = torch.max(output, 1)

    return class_names[pred.item()]

# Test the function (pick the first image from the 'paper' folder if available)
paper_dir = os.path.join(DATA_DIR, 'paper')
if os.path.isdir(paper_dir) and len(os.listdir(paper_dir)) > 0:
    test_img_path = os.path.join(paper_dir, os.listdir(paper_dir)[0])
    prediction = predict_image(model, test_img_path)
    print(f"Model prediction for {test_img_path}: {prediction}")
else:
    print('No images found in', paper_dir)

Model prediction for c:\Users\mohdy\AppData\Local\Microsoft\Windows\INetCache\IE\216KCI1W\dataset\paper\04l5I8TqdzF9WDMJ.png: paper


### Step 8: Play the Game!

This code is complete. If your model is trained, you can run this cell to have the model play against itself.

In [11]:
import random
import os

def pick_random_image(class_name):
    folder = os.path.join(DATA_DIR, class_name)
    files = os.listdir(folder)
    img = random.choice(files)
    return os.path.join(folder, img)

def rps_winner(move1, move2):
    if move1 == move2:
        return "Draw"

    rules = {
        "rock": "scissors",
        "paper": "rock",
        "scissors": "paper"
    }

    if rules[move1] == move2:
        return f"Player 1 wins! {move1} beats {move2}"
    else:
        return f"Player 2 wins! {move2} beats {move1}"


# -----------------------------------------------------------
# 1. Choose any two random classes
# -----------------------------------------------------------

choices = ["rock", "paper", "scissors"]
c1 = random.choice(choices)
c2 = random.choice(choices)

img1_path = pick_random_image(c1)
img2_path = pick_random_image(c2)

print("Randomly selected images:")
print("Image 1:", img1_path)
print("Image 2:", img2_path)


# -----------------------------------------------------------
# 2. Predict their labels using the model
# -----------------------------------------------------------

p1 = predict_image(model, img1_path)
p2 = predict_image(model, img2_path)

print("\nPlayer 1 shows:", p1)
print("Player 2 shows:", p2)

# -----------------------------------------------------------
# 3. Decide the winner
# -----------------------------------------------------------

print("\nRESULT:", rps_winner(p1, p2))

Randomly selected images:
Image 1: c:\Users\mohdy\AppData\Local\Microsoft\Windows\INetCache\IE\216KCI1W\dataset\paper\MLURv7NBoSPK4f1h.png
Image 2: c:\Users\mohdy\AppData\Local\Microsoft\Windows\INetCache\IE\216KCI1W\dataset\rock\mJ0RjteYaLfYLE9P.png

Player 1 shows: paper
Player 2 shows: rock

RESULT: Player 1 wins! paper beats rock
